In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.float_format', '{:,.4f}'.format)

# TRS

In [4]:
def TRS(n_bonds, coupons, initial_prices, final_prices, fixed_rate, t, pay_total_return, v_nominal):
    
     initial_port_value = np.dot(n_bonds, initial_prices)
     final_port_value = np.dot(n_bonds, final_prices)
     coupon_pmt = np.sum(n_bonds * coupons/t * v_nominal)
     total_return = (final_port_value - initial_port_value) + coupon_pmt
     rate = fixed_rate/t * initial_port_value

     return print(f'${(rate - total_return if pay_total_return else total_return - rate)[0]:,.2f}')

In [5]:
n_bonds = np.array([104_000, 96_000, 85_000])
coupons = np.array([0.045, 0.052, 0.060])
initial_prices = np.array([102.45, 98.75, 101.30])
final_prices = np.array([101.90, 99.40, 100.80])
fixed_rate = np.array([0.068])
t = np.array([1])
pay_total_return = True
v_nominal = np.array([100, 100, 100])

TRS(n_bonds, coupons, initial_prices, final_prices, fixed_rate, t, pay_total_return, v_nominal)

$514,780.40


# CDO

In [6]:
df = pd.read_csv('losses (6).csv')

ecl = 5_951_000
std = 4_484_600
var = 20_968_000
cvar = 23_809_000 

ucl = var - ecl

port = 118_553_188
equity = ecl * 0.6
mezanine = ecl * 0.8
junior = ecl * 1.1

senior = port - ecl - equity - mezanine - junior

lim_ecl = ecl
lim_equity = ecl + equity
lim_mezanine = ecl + equity + mezanine
lim_junior = ecl + equity + mezanine + junior
lim_senior = ecl + equity + mezanine + junior + senior

maximo = df.max().values[0]
minimo = df.min().values[0]
rango = maximo - minimo

df_grupos = pd.DataFrame({
    'grupos': list(range(1, 101)),
})
df_grupos['perdida'] = minimo + rango/99 * df_grupos.index

losses = df['losses'].to_numpy()
bins = df_grupos['perdida'].to_numpy()

frecuencias = []

for i, limite in enumerate(bins):
    if i == 0:
        count = np.sum(losses <= limite)
    else:
        count = np.sum((losses > bins[i-1]) & (losses <= limite))
    frecuencias.append(count)

df_grupos['frecuencia'] = frecuencias
df_grupos['prob_marginal'] = df_grupos['frecuencia'] / df_grupos['frecuencia'].sum()
df_grupos['prob_acumulada'] = df_grupos['prob_marginal'].cumsum()

df_filtered = df_grupos[df_grupos['perdida'] < lim_ecl]
df_extra = df_grupos[df_grupos['perdida'] >= lim_ecl].head(1)
df_ecl = pd.concat([df_filtered, df_extra])

df_filtered_equity = df_grupos[(df_grupos['perdida'] > df_ecl['perdida'].iloc[-1]) & (df_grupos['perdida'] < lim_equity)]
df_extra_equity = df_grupos[df_grupos['perdida'] >= lim_equity].head(1)
df_equity = pd.concat([df_filtered_equity, df_extra_equity])

df_filtered_mezanine = df_grupos[(df_grupos['perdida'] > df_equity['perdida'].iloc[-1]) & (df_grupos['perdida'] < lim_mezanine)]
df_extra_mezanine = df_grupos[df_grupos['perdida'] >= lim_mezanine].head(1)
df_mezanine = pd.concat([df_filtered_mezanine, df_extra_mezanine])

df_filtered_junior = df_grupos[(df_grupos['perdida'] > df_mezanine['perdida'].iloc[-1]) & (df_grupos['perdida'] < lim_junior)]
df_extra_junior = df_grupos[df_grupos['perdida'] >= lim_junior].head(1)
df_junior = pd.concat([df_filtered_junior, df_extra_junior])

df_filtered_senior = df_grupos[(df_grupos['perdida'] > df_junior['perdida'].iloc[-1]) & (df_grupos['perdida'] < lim_senior)]
df_extra_senior = df_grupos[df_grupos['perdida'] >= lim_senior].head(1)
df_senior = pd.concat([df_filtered_senior, df_extra_senior])

equity_loss_prob = 1 - df_equity['prob_acumulada'].iloc[0]
mezanine_loss_prob = 1 - df_mezanine['prob_acumulada'].iloc[0]
junior_loss_prob = 1 - df_junior['prob_acumulada'].iloc[0]

if len(df_senior) > 0:
    senior_loss_prob = 1 - df_senior['prob_acumulada'].iloc[0]
else:
    senior_loss_prob = 0

equity_loss = np.sum((df_equity['perdida'] - df_ecl['perdida'].iloc[-1]) * df_equity['prob_marginal'])
mezanine_loss = np.sum((df_mezanine['perdida'] - df_equity['perdida'].iloc[-1]) * df_mezanine['prob_marginal'])
junior_loss = np.sum((df_junior['perdida'] - df_mezanine['perdida'].iloc[-1]) * df_junior['prob_marginal'])
senior_loss = np.sum((df_senior['perdida'] - df_junior['perdida'].iloc[-1]) * df_senior['prob_marginal'])

expected_loss = np.array([equity_loss, mezanine_loss, junior_loss, senior_loss])
port_values = np.array([equity, mezanine, junior, senior])
spread = expected_loss / port_values

df_resultados = pd.DataFrame({
    'tramo': ['equity', 'mezanine', 'junior', 'senior'],
    'probabilidad': [equity_loss_prob, mezanine_loss_prob, junior_loss_prob, senior_loss_prob],
    'pérdida esperada': expected_loss,
    'spread': spread
})
df_resultados['probabilidad'] = df_resultados['probabilidad'].apply(lambda x: '{:,.2%}'.format(x))
df_resultados['spread'] = df_resultados['spread'].apply(lambda x: '{:,.2%}'.format(x))
df_resultados

,tramo,probabilidad,pérdida esperada,spread
0,equity,35.16%,"384,670.7524",10.77%
1,mezanine,15.67%,"255,061.7670",5.36%
2,junior,5.05%,"118,302.3837",1.81%
3,senior,0.89%,"29,965.9675",0.03%
